In [1]:
import carla
import math
import random
import time
import numpy as np
import cv2

In [2]:
client = carla.Client('localhost', 2000)
world = client.get_world()

In [3]:
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

In [4]:
for sensor in bp_lib.filter('sensor'):
    print(sensor.id)

sensor.other.collision
sensor.camera.depth
sensor.camera.optical_flow
sensor.camera.normals
sensor.other.lane_invasion
sensor.camera.dvs
sensor.other.imu
sensor.other.gnss
sensor.other.obstacle
sensor.other.radar
sensor.lidar.ray_cast_semantic
sensor.lidar.ray_cast
sensor.camera.rgb
sensor.camera.semantic_segmentation
sensor.other.rss
sensor.camera.instance_segmentation


In [5]:
for vehicle in bp_lib.filter('vehicle'):
    print(vehicle.id)

vehicle.audi.a2
vehicle.nissan.micra
vehicle.audi.tt
vehicle.mercedes.coupe_2020
vehicle.bmw.grandtourer
vehicle.harley-davidson.low_rider
vehicle.ford.ambulance
vehicle.micro.microlino
vehicle.carlamotors.firetruck
vehicle.carlamotors.carlacola
vehicle.carlamotors.european_hgv
vehicle.ford.mustang
vehicle.chevrolet.impala
vehicle.lincoln.mkz_2020
vehicle.citroen.c3
vehicle.dodge.charger_police
vehicle.nissan.patrol
vehicle.jeep.wrangler_rubicon
vehicle.mini.cooper_s
vehicle.mercedes.coupe
vehicle.dodge.charger_2020
vehicle.ford.crown
vehicle.seat.leon
vehicle.toyota.prius
vehicle.yamaha.yzf
vehicle.kawasaki.ninja
vehicle.bh.crossbike
vehicle.mitsubishi.fusorosa
vehicle.tesla.model3
vehicle.gazelle.omafiets
vehicle.tesla.cybertruck
vehicle.diamondback.century
vehicle.mercedes.sprinter
vehicle.audi.etron
vehicle.volkswagen.t2
vehicle.lincoln.mkz_2017
vehicle.dodge.charger_police_2020
vehicle.vespa.zx125
vehicle.mini.cooper_s_2021
vehicle.nissan.patrol_2021
vehicle.volkswagen.t2_2021


In [6]:
vehicle_bp = world.get_blueprint_library().filter('*cybertruck*')
vehicle = world.try_spawn_actor(vehicle_bp[0], spawn_points[0])

In [7]:
CAMERA_POS_Z = 3
CAMERA_POS_X = 0

camera_bp = bp_lib.find('sensor.camera.rgb')
camera_init_trans = carla.Transform(carla.Location(z = CAMERA_POS_Z, x = CAMERA_POS_X))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

In [8]:
sem_camera_bp = bp_lib.find('sensor.camera.semantic_segmentation')
sem_camera = world.spawn_actor(sem_camera_bp, camera_init_trans, attach_to=vehicle)

In [9]:
inst_camera_bp = bp_lib.find('sensor.camera.instance_segmentation')
inst_camera = world.spawn_actor(inst_camera_bp, camera_init_trans, attach_to = vehicle)

In [10]:
depth_camera_bp = bp_lib.find('sensor.camera.depth')
depth_camera = world.spawn_actor(depth_camera_bp, camera_init_trans, attach_to = vehicle)

In [11]:
dvs_camera_bp = bp_lib.find('sensor.camera.dvs')
dvs_camera = world.spawn_actor(dvs_camera_bp, camera_init_trans, attach_to = vehicle)

In [12]:
opt_camera_bp = bp_lib.find('sensor.camera.dvs')
opt_camera = world.spawn_actor(opt_camera_bp, camera_init_trans, attach_to = vehicle)

In [13]:
def rgb_callback(image, data_dict):
    data_dict['rgb_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

In [14]:
def sem_callback(image, data_dict):
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

In [15]:
def inst_callback(image, data_dict):
    data_dict['inst_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

In [16]:
def depth_callback(image, data_dict):
    image.convert(carla.ColorConverter.LogarithmicDepth)
    data_dict['depth_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

In [17]:
def opt_callback(data, data_dict):
    image = data.get_color_coded_flow()
    img = np.reshape(np.color(image.raw_data), (image.height, image.width, 4))
    img[:,:,3] = 255
    data_dict['opt_image'] = img

In [18]:
def dvs_callback(data, data_dict):
    dvs_events = np.frombuffer(data.raw_data, dtype =np.dtype([
        ('x', np.uint16), ('y', np.uint16), ('t', np.int64), ('pol', bool)]))
    data_dict['dvs_image'] = np.zeros((data.height, data.width, 4), dtype=np.uint8)
    dvs_img = np.zeros((data.height, data.width, 3), dtype=np.uint8)
    dvs_img[dvs_events[:]['y'], dvs_events[:]['x'], dvs_events[:]['pol'] * 2 ] = 255
    data_dict['dvs_image'][:, :, 0:3] = dvs_img

In [19]:
def rgb_callback(image, data_dict):
    data_dict['rgb_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

def sem_callback(image, data_dict):
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

def inst_callback(image, data_dict):
    data_dict['inst_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

def depth_callback(image, data_dict):
    image.convert(carla.ColorConverter.LogarithmicDepth)
    data_dict['depth_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

'''def opt_callback(data, data_dict):
    image = data.get_color_coded_flow()
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    img[:,:,3] = 255
    data_dict['opt_image'] = img'''

def dvs_callback(data, data_dict):
    dvs_events = np.frombuffer(data.raw_data, dtype=np.dtype([
        ('x', np.uint16), ('y', np.uint16), ('t', np.int64), ('pol', bool)]))
    data_dict['dvs_image'] = np.zeros((data.height, data.width, 4), dtype=np.uint8)
    dvs_img = np.zeros((data.height, data.width, 3), dtype=np.uint8)
    dvs_img[dvs_events[:]['y'], dvs_events[:]['x'], dvs_events[:]['pol'] * 2] = 255
    data_dict['dvs_image'][:,:,0:3] = dvs_img


In [20]:
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()

In [23]:
vehicle.set_autopilot(True)

In [21]:
sensor_data = {'rgb_image': np.zeros((image_h, image_w, 4)),
               'sem_image': np.zeros((image_h, image_w, 4)),
               'depth_image': np.zeros((image_h, image_w, 4)),
               'dvs_image': np.zeros((image_h, image_w, 4)),
               'opt_image': np.zeros((image_h, image_w, 4)),
               'inst_image': np.zeros((image_h, image_w, 4))}

#cv2.namedWindow('All cameras', cv2.WINDOW_AUTOSIZE)

camera.listen(lambda image: rgb_callback(image, sensor_data))
sem_camera.listen(lambda image: sem_callback(image, sensor_data))
inst_camera.listen(lambda image: inst_callback(image, sensor_data))
depth_camera.listen(lambda image: depth_callback(image, sensor_data))
dvs_camera.listen(lambda image: dvs_callback(image, sensor_data))
#opt_camera.listen(lambda image: opt_callback(image, sensor_data))

while True:
    top_row = np.concatenate((sensor_data['rgb_image'], sensor_data['sem_image'], sensor_data['inst_image']), axis=1)
    lower_row = np.concatenate((sensor_data['depth_image'], sensor_data['dvs_image'], sensor_data['dvs_image']), axis=1)
    tiled = np.concatenate((top_row, lower_row), axis=0)

    cv2.imshow('All cameras', tiled)

    if cv2.waitKey(1) == ord('q'):
        break
        
camera.stop()
sem_camera.stop()
inst_camera.stop()
depth_camera.stop()
dvs_camera.stop()
opt_camera.stop()
cv2.destroyAllWindows()
        
                         